### Import Packages

In [2]:
import utils 
from SDA_layers import StackedDA 
import os
from sklearn.cross_validation import train_test_split
from sklearn import preprocessing
from sklearn import linear_model
import numpy as np
import pandas as pd
from sklearn import linear_model
from sklearn.metrics import accuracy_score
from sklearn.linear_model import Ridge

### Import Dataset

In [3]:
data = pd.read_csv("final_dataset.csv")
patient = data['PatientID']
y = data['Cancer_Type']
X = data.drop(['Cancer_Type','PatientID'],axis=1)



### Test Train Split

In [4]:
X = data.drop(['Cancer_Type','PatientID'],axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.10, random_state=42)
le = preprocessing.LabelEncoder()
le.fit(y_train) 
y_train = pd.DataFrame(le.transform(y_train))
y_val = pd.DataFrame(le.transform(y_val))
y_test = pd.DataFrame(le.transform(y_test))


### Validation to determine alpha

In [5]:
regr = linear_model.Ridge(max_iter=10000)
alphas = np.logspace(-4, -1, 10)
alphas

array([ 0.0001    ,  0.00021544,  0.00046416,  0.001     ,  0.00215443,
        0.00464159,  0.01      ,  0.02154435,  0.04641589,  0.1       ])

In [7]:
scores = [regr.set_params(alpha=alpha).fit(X_train, y_train).score(X_val, y_val) for alpha in alphas]
best_alpha = alphas[scores.index(max(scores))]


In [8]:
scores

[0.91260087912782206,
 0.9130759043612009,
 0.91403180846601451,
 0.91583255397762975,
 0.91886576671663167,
 0.92319766610472609,
 0.92820658587487181,
 0.93276232710486107,
 0.93599554017474873,
 0.93774584208296141]

In [9]:
best_alpha

0.10000000000000001

### Testing

In [10]:
regr = linear_model.Ridge(alpha=best_alpha, max_iter=10000)
model = regr.fit(X_train,y_train)
prediction = model.predict(X_test)
y = y_test[0].as_matrix()
print(accuracy_score(y,prediction.astype(int)))


0.471849865952


In [11]:
from sklearn.metrics import classification_report
print ("Classification Report \n {}".format(classification_report(y, prediction.astype(int))))

Classification Report 
              precision    recall  f1-score   support

          0       0.46      1.00      0.63        36
          1       0.78      0.40      0.53        80
          2       0.65      0.59      0.62        29
          3       0.05      0.17      0.07         6
          4       0.37      0.47      0.41        32
          5       0.42      0.49      0.45        45
          6       0.51      0.52      0.51        52
          7       0.25      0.12      0.17        32
          8       0.26      0.33      0.29        21
          9       1.00      0.38      0.55        40
         10       0.00      0.00      0.00         0

avg / total       0.56      0.47      0.47       373



C:\Users\Swati\AppData\Local\Continuum\Anaconda2\lib\site-packages\sklearn\metrics\classification.py:1076: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
